### Imports

In [3]:
# Imports
import os
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from skmultilearn.adapt import MLARAM, MLkNN
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
from scipy.stats import mode


In [2]:
!pip install scikit-multilearn

### Load Data

In [ ]:
# Load extracted features and labels
root_dir = "Extracted_Features"
train_features = np.load(os.path.join(root_dir, "ResNet50_train_features.npy"))
train_labels = np.load(os.path.join(root_dir, "ResNet50_train_labels.npy"))
test_features = np.load(os.path.join(root_dir, "ResNet50_test_features.npy"))
test_labels = np.load(os.path.join(root_dir, "ResNet50_test_labels.npy"))

print("Data loaded successfully.")
print(f"Train Features: {train_features.shape}, Train Labels: {train_labels.shape}")
print(f"Test Features: {test_features.shape}, Test Labels: {test_labels.shape}")

### Binary Relevance Classifier

In [ ]:
# Binary Relevance Classifier
def train_binary_relevance(train_features, train_labels, test_features, test_labels):
    n_classes = train_labels.shape[1]
    classifiers = []
    predictions = []

    for i in range(n_classes):
        print(f"Training Binary Classifier for Class {i+1}/{n_classes}...")
        clf = LogisticRegression()
        clf.fit(train_features, train_labels[:, i])
        classifiers.append(clf)
        pred = clf.predict(test_features)
        predictions.append(pred)

    # Combine predictions for multi-label output
    predictions = np.array(predictions).T
    accuracy = accuracy_score(test_labels, predictions)
    print(f"Binary Relevance Classifier Accuracy: {accuracy:.4f}")
    return classifiers, predictions

binary_classifiers, binary_predictions = train_binary_relevance(train_features, train_labels, test_features, test_labels)

### MLARAM

In [ ]:
# MLARAM Classifier
def train_mlaram(train_features, train_labels, test_features, test_labels):
    print("Training MLARAM Model...")
    mlaram = MLARAM()
    mlaram.fit(train_features, train_labels)
    predictions = mlaram.predict(test_features).toarray()
    accuracy = accuracy_score(test_labels, predictions)
    print(f"MLARAM Accuracy: {accuracy:.4f}")
    return mlaram, predictions

mlaram_model, mlaram_predictions = train_mlaram(train_features, train_labels, test_features, test_labels)

### Multi-label SVM

In [ ]:
# Multi-label SVM
def train_multi_label_svm(train_features, train_labels, test_features, test_labels):
    print("Training Multi-label SVM...")
    svm = BinaryRelevance(classifier=SVC(probability=True))
    svm.fit(train_features, train_labels)
    predictions = svm.predict(test_features).toarray()
    accuracy = accuracy_score(test_labels, predictions)
    print(f"Multi-label SVM Accuracy: {accuracy:.4f}")
    return svm, predictions

svm_model, svm_predictions = train_multi_label_svm(train_features, train_labels, test_features, test_labels)

### Multi-label KNN

In [ ]:
# Multi-label KNN
def train_multi_label_knn(train_features, train_labels, test_features, test_labels, k=5):
    print("Training Multi-label KNN...")
    knn = MLkNN(k=k)
    knn.fit(train_features, train_labels)
    predictions = knn.predict(test_features).toarray()
    accuracy = accuracy_score(test_labels, predictions)
    print(f"Multi-label KNN Accuracy: {accuracy:.4f}")
    return knn, predictions

knn_model, knn_predictions = train_multi_label_knn(train_features, train_labels, test_features, test_labels)


### Multi-label SVM

In [ ]:
# Multi-label SVM
def train_multi_label_svm(train_features, train_labels, test_features, test_labels):
    print("Training Multi-label SVM...")
    svm = BinaryRelevance(classifier=SVC(probability=True))
    svm.fit(train_features, train_labels)
    predictions = svm.predict(test_features).toarray()
    accuracy = accuracy_score(test_labels, predictions)
    print(f"Multi-label SVM Accuracy: {accuracy:.4f}")
    return svm, predictions

svm_model, svm_predictions = train_multi_label_svm(train_features, train_labels, test_features, test_labels)


### Multi-label KNN

In [ ]:
# Multi-label KNN
def train_multi_label_knn(train_features, train_labels, test_features, test_labels, k=5):
    print("Training Multi-label KNN...")
    knn = MLkNN(k=k)
    knn.fit(train_features, train_labels)
    predictions = knn.predict(test_features).toarray()
    accuracy = accuracy_score(test_labels, predictions)
    print(f"Multi-label KNN Accuracy: {accuracy:.4f}")
    return knn, predictions

knn_model, knn_predictions = train_multi_label_knn(train_features, train_labels, test_features, test_labels)


### Majority Voting

In [ ]:
# Majority Voting
def majority_voting(*predictions):
    combined_predictions = mode(predictions, axis=0)[0].squeeze()
    return combined_predictions

combined_majority = majority_voting(binary_predictions, mlaram_predictions, svm_predictions, knn_predictions)
accuracy_majority = accuracy_score(test_labels, combined_majority)
print(f"Majority Voting Accuracy: {accuracy_majority:.4f}")

### Weighted Averaging


In [ ]:
# Weighted Averaging
def weighted_averaging(*predictions, weights):
    weighted_sum = np.zeros_like(predictions[0], dtype=float)
    for pred, weight in zip(predictions, weights):
        weighted_sum += weight * pred
    final_predictions = (weighted_sum > 0.5).astype(int)
    return final_predictions

weights = [0.3, 0.2, 0.3, 0.2]  # Example weights for the models
combined_weighted = weighted_averaging(binary_predictions, mlaram_predictions, svm_predictions, knn_predictions, weights=weights)
accuracy_weighted = accuracy_score(test_labels, combined_weighted)
print(f"Weighted Averaging Accuracy: {accuracy_weighted:.4f}")

### Summary

In [ ]:
print("Summary of Model Accuracies:")
print(f"Binary Relevance Classifier Accuracy: {accuracy_score(test_labels, binary_predictions):.4f}")
print(f"MLARAM Accuracy: {accuracy_score(test_labels, mlaram_predictions):.4f}")
print(f"Multi-label SVM Accuracy: {accuracy_score(test_labels, svm_predictions):.4f}")
print(f"Multi-label KNN Accuracy: {accuracy_score(test_labels, knn_predictions):.4f}")
print(f"Majority Voting Accuracy: {accuracy_majority:.4f}")
print(f"Weighted Averaging Accuracy: {accuracy_weighted:.4f}")